In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import string
from matplotlib.collections import LineCollection
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time
import scipy as sp

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('figure', **{'dpi' : 200})

# 1. 

Возьмем корпуса "Войны и мира" на русском и английском и посчитаем на них частоты букв. Будем разделять языки - мы не хотим, чтобы при расшифровке в словах оказывались буквы из разных языков. 

In [18]:
def preprocessing(text):
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    text = re.sub(r'\n{1,}', ' ', text)
    text = re.sub(r'\r{1,}', ' ', text)
    text = re.sub(r'\t{1,}', ' ', text)
    text = re.sub(r'[–—]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r' {1,}', ' ', text)
    return text

with open('WarAndPeace.txt', 'r') as fin:
    corpus_ru = fin.read()
    corpus_ru = preprocessing(corpus_ru)

with open('WarAndPeaceEng.txt', 'r') as fin:
    corpus_en = fin.read()
    corpus_en = preprocessing(corpus_en)

In [40]:
ru = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
en = 'abcdefghijklmnopqrstuvwxyz '

def count_freq(text, n, lang=ru):
    if n == 1:
        ans = Counter([a for a in text if a in lang])
    elif n == 2:
        ans = Counter([text[i:i + n] for i in range(len(text) - n + 1) if (text[i:i + n][0] in lang and text[i:i + n][1] in lang)])
    else:
        raise NotImplementedError()
    return ans

In [41]:
letter_freq_ru = count_freq(corpus_ru, n=1, lang=ru)
letter_freq_en = count_freq(corpus_en, n=1, lang=en)

In [81]:
letter_freq_ru

Counter({' ': 109517,
         'в': 24824,
         'о': 61282,
         'й': 6210,
         'н': 35119,
         'а': 45209,
         'и': 35838,
         'м': 15940,
         'р': 24570,
         'с': 28128,
         'ы': 10233,
         'з': 9602,
         'е': 42519,
         'т': 30619,
         'л': 27277,
         'ь': 10498,
         'к': 19328,
         'ч': 7349,
         'г': 11177,
         'д': 16387,
         'у': 15454,
         'п': 13847,
         'я': 12477,
         'ж': 5460,
         'б': 9310,
         'щ': 1514,
         'ф': 1209,
         'э': 1629,
         'х': 4600,
         'ю': 3495,
         'ш': 5090,
         'ц': 2179,
         'ё': 431,
         'ъ': 283})

In [42]:
def make_encryption(text):
    text = preprocessing(text)
    alphabet = list(set(text))
    shuffled_alphabet = alphabet.copy()
    random.shuffle(shuffled_alphabet)
    encryption_rule = dict(zip(alphabet, shuffled_alphabet))
    return ''.join([encryption_rule[a] for a in text])

Возьмем несколько абзацев текста из русскоязычной и англоязычной википедии и зашифруем их. Для расшифровки будем считать частоту вхождения униграмм и сопоставлять с частотой вхождения этих же униграмм в наши корпуса. Предположение: частоты встречаемости одинаковых букв будут пропорциональны.

В английском языке меньше букв, чем в русском, значит найти соответствие будет проще. Можем сразу предположить, что качество расшифровки русских сообщений будет хуже.

In [43]:
test = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'


test_ru = '''Матрёшка — русская деревянная игрушка в виде расписной куклы, внутри которой находятся подобные ей куклы меньшего размера.
Число вложенных кукол — обычно три и более. Обычно имеют форму яйца с плоским донцем и состоят из двух разъёмных частей, верхней и нижней.
На традиционных матрёшках изображена женщина в красном сарафане и платке. В наше время темы для росписи разнообразны: сказочные персонажи,
девушки, а также семьи. Стали нередки и матрёшки пародийного характера с изображением политических деятелей. Сравнительно недавно стала
набирать популярность матрёшка с изображением портрета — портретная матрёшка. Сейчас матрёшки делают в различных мастерских.
Сначала подбирают подходящий вид древесины. Из-за мягкости в основном выбирают липу, реже ольху или берёзу. Деревья обычно
срубают ранней весной, снимают кору, но не полностью, чтобы во время сушки древесина не давала трещин. Затем брёвна складируют и сушат
в течение нескольких лет в хорошо вентилируемом месте. Древесина может заготавливаться в регионе изготовления или быть завозной.
К обработке древесины необходимо приступать тогда, когда она не сухая, но и не сырая. Каждая заготовка проходит более десятка операций.
Самую маленькую куклу делают первой.
Когда матрёшка готова, приступают к следующей фигурке, в которую войдёт первая. Заготовка необходимой высоты обрабатывается
и разрезается на верхнюю и нижнюю части. Первой делается нижняя часть. Затем удаляют древесину изнутри обеих частей второй куклы так,
чтобы меньшая кукла плотно вставлялась внутрь. Потом процесс повторяется для куклы большего размера, в которую войдут две предыдущие и т. д.
Количество кукол может быть различным. В заключение процесса каждую куклу покрывают масляным лаком.
После окончательной сушки и полировки художник приступает к раскраске. В качестве красок используется акварель, гуашь, темпера,
реже масляные краски. Несмотря на разнообразие красок, мастера по-прежнему отдают предпочтение гуаши.'''

encrypted_test_ru = make_encryption(test_ru)

In [44]:
test_en = '''Coffee is a brewed drink prepared from roasted coffee beans, the seeds of berries from certain Coffea species.
When coffee berries turn from green to bright red in color – indicating ripeness – they are picked, processed, and dried.
Dried coffee seeds (referred to as "beans") are roasted to varying degrees, depending on the desired flavor.
Roasted beans are ground and then brewed with near-boiling water to produce the beverage known as coffee.
Coffee is darkly colored, bitter, slightly acidic and has a stimulating effect in humans, primarily due to its caffeine content.
It is one of the most popular drinks in the world, and can be prepared and presented in a variety of ways.
It is usually served hot, although chilled or iced coffee is common. Sugar, sugar substitutes, milk or cream are often
used to lessen the bitter taste. It may be served with coffee cake or another sweet dessert like doughnuts.
A commercial establishment that sells prepared coffee beverages is known as a coffee shop
(not to be confused with Dutch coffeeshops selling cannabis).
Clinical research indicates that moderate coffee consumption is benign or mildly beneficial as a stimulant in healthy adults,
with continuing research on whether long-term consumption reduces the risk of some diseases, although some of the long-term
studies are of questionable credibility.'''

encrypted_test_en = make_encryption(test_en)

In [45]:
print('RU EXAMPLE: ' + encrypted_test_ru + '\n')
print('EN EXAMPLE: ' + encrypted_test_en)

RU EXAMPLE:  ъщфсёйъшфнььйъяшвафауяччъяшорфнёйъшушуовашфъьгоьчждшйнйюишучнщфошйжщжфждшчъежвящьяшгжвжзчиашадшйнйюиш ачлёаржшфът афъшбоьюжшуюжцаччиешйнйжюшжзибчжшщфошошзжюаашжзибчжшо амщшхжф ншядыъшьшгюжьйо швжчыа шошьжьщжящшотшвунешфътпс чиешбъьщадшуафечадшошчоцчадшчъшщфъвоыожччиеш ъщфсёйъешотжзфъцачъшцачкочъшушйфъьчж шьъфъхъчашошгюъщйашушчъёашуфа яшща ишвюяшфжьгоьошфътчжжзфътчишьйътжбчиашгафьжчъцошваунёйошъшщъйцашьа лошьщъюошчафавйошош ъщфсёйошгъфжводчжржшеъфъйщафъшьшотжзфъцачоа шгжюощобаьйоешваящаюадшьфъучощаюлчжшчавъучжшьщъюъшчъзофъщлшгжгнюяфчжьщлш ъщфсёйъшьшотжзфъцачоа шгжфщфащъшгжфщфащчъяш ъщфсёйъшьадбъьш ъщфсёйошваюъмщшушфътюобчиеш ъьщафьйоешьчъбъюъшгжвзофъмщшгжвежвякодшуовшвфауаьочишоттъш ярйжьщошушжьчжучж шуизофъмщшюогншфацашжюленшоюошзафстншвафауляшжзибчжшьфнзъмщшфъччадшуаьчждшьчо ъмщшйжфншчжшчашгжючжьщлмшбщжзишужшуфа яшьнёйошвфауаьочъшчашвъуъюъшщфакочштъща шзфсучъшьйюъвофнмщшошьнёъщшушщабачоашчаьйжюлйоешюащшушежфжёжшуачщоюофна ж ш аьщашвфауаьочъш жцащштържщъуюоуъщльяшушфарожча

In [46]:
def make_decryption(text, frequences, n, lang=ru, real_text=None):
    new_freqs = count_freq(text, n, lang)
    
    length = len(new_freqs)
    if length > len(frequences):
        return 'Size of the alphabet is smaller than amount letters in test text.'
    new_freqs = new_freqs.most_common()
    frequences = frequences.most_common(length)
    decryption_rule = dict(
                                zip(
                                        [k for k, v in new_freqs],
                                        [k for k, v in frequences]
                                    )
                            )
    if n == 1:
        ans = ''.join([decryption_rule[a] for a in text])
    elif n == 2:
        bigrams = [text[i:i + n] for i in range(0, len(text) - n + 1, 2)]
        ans = ''.join([decryption_rule[a] for a in bigrams])
    else:
        raise NotImplementedError()
        
    if real_text:
        real_text = preprocessing(real_text)
        if np.array(list(ans)).shape != np.array(list(real_text)).shape:
            real_text = real_text[:-1]
        print(f'Accuracy = {round(sum(np.array(list(ans)) == np.array(list(real_text))) / len(list(ans)), 3)}')
    return ans

In [47]:
make_decryption(encrypted_test_ru, letter_freq_ru, 1, ru, test_ru)

Accuracy = 0.196


'уоницшво идллвоя маиаряссоя тюидшво р ртма иолптлсез вдвкг рсднит венеиез сойемянля пемеысга аз вдвкг уасчшаюе иоьуаио жтлке ркехассгй вдвек еыгжсе нит т ыекаа еыгжсе туабн феиуд язэо л пкелвту месэау т лелнеян ть мрдй иоьёцусгй жолназ раийсаз т стхсаз со ниомтэтессгй уоницшвой тьеыиохасо хасщтсо р виолсеу лоиофоса т пконва р соша риауя науг мкя иелптлт иоьсееыиоьсг лвоьежсга паилесохт мардшвт о новха лаучт лнокт саиамвт т уоницшвт поиемтзсеюе йоиовнаио л тьеыиохастау пектнтжалвтй маянаказ лиорстнакчсе саморсе лноко соытионч пепдкяиселнч уоницшво л тьеыиохастау пеиниано пеиниансоя уоницшво лазжол уоницшвт макобн р иоьктжсгй уолнаилвтй лсожоко пемытиобн пемйемящтз ртм миаралтсг тььо уяювелнт р елсерсеу ргытиобн ктпд иаха екчйд ткт ыаицьд маиарчя еыгжсе лидыобн иоссаз ралсез лстуобн веид се са пекселнчб жнеыг ре риауя лдшвт миаралтсо са мороко ниащтс ьонау ыицрсо лвкомтидбн т лдшон р нажаста салвекчвтй кан р йеиеше раснтктидауеу уална миаралтсо уехан ьоюенорктрончля р иаютеса тьюенеркас

In [48]:
make_decryption(encrypted_test_en, letter_freq_en, 1, en, test_en)

Accuracy = 0.305


'dillee nt h foeper ronsv yoeyhoer loiw oihtaer dillee fehst aue teert il feoonet loiw deoahns dilleh tyednet pues dillee feoonet acos loiw goees ai fongua oer ns dimio nsrndhansg onyesett aueb hoe yndver yoidetter hsr roner roner dillee teert oeleooer ai ht fehst hoe oihtaer ai khobnsg regoeet reyesrnsg is aue retnoer lmhkio oihtaer fehst hoe goicsr hsr aues foeper pnau sehofinmnsg phaeo ai yoircde aue fekeohge vsips ht dillee dillee nt rhovmb dimioer fnaaeo tmnguamb hdnrnd hsr uht h tanwcmhansg elleda ns ucwhst yonwhonmb rce ai nat dhllense disaesa na nt ise il aue wita yiycmho ronsvt ns aue piomr hsr dhs fe yoeyhoer hsr yoetesaer ns h khoneab il phbt na nt ctchmmb teoker uia hmauicgu dunmmer io nder dillee nt diwwis tcgho tcgho tcftanacaet wnmv io doehw hoe ilaes cter ai mettes aue fnaaeo ahtae na whb fe teoker pnau dillee dhve io hsiaueo tpeea retteoa mnve ricguscat h diwweodnhm etahfmntuwesa auha temmt yoeyhoer dillee fekeohget nt vsips ht h dillee tuiy sia ai fe dislcter pnau rca

Можем заметить, что наш подход достаточно хорошо находит пробелы в тексте и еще некоторые часто встречающиеся буквы. Но для расшифровки этого еще недостаточно. Также мы верно предположили, что качество на английском языке будет заметно лучше.

# 2.

Проделаем то же самое для биграмм:

In [49]:
bigram_freq_ru = count_freq(corpus_ru, 2, ru)
bigram_freq_en = count_freq(corpus_en, 2, en)

In [86]:
bigram_freq_ru.most_common(15)

[('о ', 13219),
 ('и ', 11355),
 ('а ', 10485),
 ('е ', 9971),
 (' с', 9847),
 (' п', 9750),
 (' в', 9591),
 (' н', 9300),
 ('то', 8491),
 (' о', 7671),
 ('я ', 7043),
 (' к', 7026),
 (' и', 6818),
 ('ст', 6671),
 ('ь ', 6548)]

In [84]:
print(f'Amount of ru bigram: {len(bigram_freq_ru)}\nAmount of en bigram: {len(bigram_freq_en)}')

Amount of ru bigram: 794
Amount of en bigram: 602


Теперь наши словари частот слишком большие, в тестовых текстах биграмм будет гораздо меньше, что затруднит расшифровку.

In [50]:
make_decryption(encrypted_test_ru, bigram_freq_ru, 2, ru, test_ru)

Accuracy = 0.074


'олраойроколюривестал зоков нр ы в и отвшатконелыруел вскейа кнрае  тертенаовсяажс  нл икрьизвуна гогревссиглазкопрвсо быетыла ыл чслэт вскосламочекарае е ча е пй ожь товсу чианияаюайи  л я улионикушлотоя  уерботопипабакопрбиачэтбынем наванд бнае оркиод ии рамнапжислэтпоноомв бртоадемимени  чсуеси оттрнеь онкуо пл бто с мзн потовгла  култьлоред  нте жете о созьемпрния роадчеиз саливовучст зселиывтьма чя лоиря чтза иалпелитопоноомв е бетеннздшикаебо сёали  лмий о  чорло сосрытуволиилатбоегодя о обрыегехка басобкагодии овицо же сиййсщиь гок олраойроя тоадемиментионл ьн кчт санратаов нолраойроя одль лолраойлистег р оото уютуниилолгоалриляя овль м ст ицо у  ст сяажтвнавшекие звоесремиакпоё  тгое от уь об эа ояам р озажнкоят посянн усе смомашстал зчуламочекагаге р оо слода воь наруну р о ты  ика б сось гонцбыермоа каькло нидв е ие звоеси  бстмедии рацаесвинолодоомоби ри мнны у тоя сеноа тьытенти иво тнялиил е оотсятеыша енудзаы ло эповом ст кваисовпошетавилс дмезасьжедаа кодужи бтоаф дю  ео

In [51]:
make_decryption(encrypted_test_en, bigram_freq_en, 2, en, test_en)

Accuracy = 0.109


't atr n  tleunulthngouni m ars ad ich aspe cthhe s ie g to tine edth t sy  oouanutf hol ghifert atorndsashanteonert atr y  oouan htuerich ry ais h runw frasthn ert elhi w nroarstasivisan tinags  a mneeed ofepanedd tod admed admed t atr ndr yoasur o ad a s  tg to tree f eshad a trreokstngga aanngomis nstenerine m ki ad spewntaspe cthy orris  aryf efd tod inisy  aaid iminnaorp extistteve o h rofevi e ine g owfobumi gtss  tt atr he s ie  fngreopset el ad ssvi ondticotlsermianes  pliess ndarpootarstpaatiro  wlietto tofdirellsewae a n wehev  i we t  liso itn  t be  s hon ecko yeciothiad wakn erine rnbyd tod roerg  m ars ad tod ofanishad  ws trremerteneaecaun o  ffecedewhnd oowd cho deinlycohedasithenhinethhe s ie  fheh laercetthicetthicedsarraha trdquenhiigorhoree  shaeremth h rldwiis hony ithahiie ce it eamy e edwothteit dt atr hesce nts  gin ondaioonganedghuspte awbeegov tlet av oshdepa c yticata l hpro edsi tofomrethhe s ie g owfobu t fmi gtss  tlet atr ndchft go a y e t nyemthteit dwac

Действительно, результат получился заметно хуже, теперь алгоритм ошибается даже на пробелах. Ну и скор это подтверждает.

# 3. 

Применим метод обучения, основанный на MCMC-сэмплировании, который бы использовал применение биграмм.

Пусть $x$ - вектор правил расшифровки.
- Возьмем начальный вектор правил расшифровки случайно сопоставив символы зашифрованного сообщения буквам из алфавита. Если различных символов меньше, чем размер алфавита, то будем брать топ букв алфавита по встречаемости.
- Далее будем сэмплировать $x$ из $q$, где $q(x, x^{(i)})$ - распределения правил, которые можно получить из правила $x^{(i)}$, поменяв местами 2 правила декодирования
- Посчитаем частоты встречаемости биграмм в сообщении после применения полученного правила дешифровки и посчитаем правдоподобие:
$$ L_x = \prod_{a_1, a_2}R(a_1a_2)^{F_x(a_1a_2)}, $$
где $R(a_1a_2)$ - количество вхождений биграммы $a_1a_2$ в обучающий текст, а $F_x(a_1a_2)$ - количество вхождений биграммы $a_1a_2$ в расшифрованное сообщение при помощи правила $x$.
- Логарифм правдоподобия (в расчетах используем его):
$$ log L_x = \sum_{a_1, a_2}F_x(a_1a_2)logR(a_1a_2) $$
- Далее с вероятностью a = $\frac{L_{x_{new}}}{L_{x_{cur}}}$ (1, если a $\geq$ 1) будем брать новое правило дешифровки.

In [87]:
def log_score(new_freqs, frequences):
    score = 0
    for k, v in new_freqs.items():
        if k in frequences.keys():
            score += v * np.log(frequences[k])
    return score

def change_decryption_rule_once(decryption_rule, lang):
    changed = decryption_rule.copy()
    i = random.randint(0, len(changed) - 1)
    j = random.randint(0, len(changed) - 1)
    keys = list(changed.keys()) 
    if i == j:
        return change_decryption_rule_once(changed, lang)
    else:
        a = changed[keys[i]]
        b = changed[keys[j]]
        changed[keys[i]] = b
        changed[keys[j]] = a
        return changed

def init_decryption_rule(text, lang):
    alphabet = list(set(text))
    if len(lang) > 30:
        most_common_letters = letter_freq_ru.most_common(len(alphabet))
    else:
        most_common_letters = letter_freq_en.most_common(len(alphabet))
    most_common_letters = [k for k,v in most_common_letters]
    decryption_rule = dict(zip(alphabet, most_common_letters))
    return decryption_rule

def metropolis_hastings_log_accept(l, l_new):
    if l_new > l:
        return True
    else:
        return random.random() < np.exp(l_new-l)

def MCMC_decryption(iterations, text, frequences, n, lang, real_text=None):
    cur_decryption_rule = init_decryption_rule(text, lang)
    decrypted_text = ''.join([cur_decryption_rule[a] for a in text])
    cur_freqs = count_freq(decrypted_text, n, lang)
    cur_score = log_score(cur_freqs, frequences)
    for i in range(iterations):
        new_decryption_rule = change_decryption_rule_once(cur_decryption_rule, lang)
        new_decrypted_text = ''.join([new_decryption_rule[a] for a in text])
        new_freqs = count_freq(new_decrypted_text, n, lang)
        new_score = log_score(new_freqs, frequences)
        
        if metropolis_hastings_log_accept(cur_score, new_score):
            cur_decryption_rule = new_decryption_rule
            decrypted_text = new_decrypted_text
            cur_score = new_score
        if i%1000 == 0:
            print(f'ITER {i}: score={round(cur_score, 3)} | {decrypted_text[:75]}')
    
    if real_text:
        real_text = preprocessing(real_text)
        print(f'Accuracy = {round(sum(np.array(list(decrypted_text)) == np.array(list(real_text))) / len(list(decrypted_text)), 3)}')
    
    return cur_decryption_rule, decrypted_text

In [92]:
decryption_rule, decrypted_text = MCMC_decryption(10000, encrypted_test_en, bigram_freq_en, 2, en, test_en)

ITER 0: score=7837.193 | cowwllpndpxpkrlyl p rnegpirlixrl pwrosproxdvl pcowwllpklxedpvtlpdll dpowpkl
ITER 1000: score=12634.927 | coffee is a mrewed drink brebared frop roasted coffee means the seeds of me
ITER 2000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 3000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 4000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 5000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 6000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 7000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 8000: score=12714.374 | coffee is a brewed drink prepared from roasted coffee beans the seeds of be
ITER 9000: score=12714.374 | coffee is a brewed drink prepa

In [93]:
decrypted_text

'coffee is a brewed drink prepared from roasted coffee beans the seeds of berries from certain coffea species when coffee berries turn from green to bright red in color indicating ripeness they are picked processed and dried dried coffee seeds referred to as beans are roasted to varying degrees depending on the desired flavor roasted beans are ground and then brewed with nearboiling water to produce the beverage known as coffee coffee is darkly colored bitter slightly acidic and has a stimulating effect in humans primarily due to its caffeine content it is one of the most popular drinks in the world and can be prepared and presented in a variety of ways it is usually served hot although chilled or iced coffee is common sugar sugar substitutes milk or cream are often used to lessen the bitter taste it may be served with coffee cake or another sweet dessert like doughnuts a commercial establishment that sells prepared coffee beverages is known as a coffee shop not to be confused with dut

In [98]:
decryption_rule, decrypted_text = MCMC_decryption(10000, encrypted_test_ru, bigram_freq_ru, 2, ru, test_ru)

ITER 0: score=5943.757 | ряёыфйгящыхжжгяущьеыеэуппяущимыхйгящэщэиьещыяжзижпндщгхготщэпхёыищгнёнындщп
ITER 1000: score=13186.94 | воктыждо трссдоя петегянноя амтрждо г гапе тосшасний дрдлу гнркта дикитий н
ITER 2000: score=13603.888 | мовтыжко трсскоя петедянноя ицтржко д дипе тосчиснай крклу днрвти каватай н
ITER 3000: score=13924.295 | матрушка рвсская бередянная ифрвшка д дибе расписной квклы днвтри которой н
ITER 4000: score=14299.751 | матрцшка русская деревянная изрушка в виде расписной куклы внутри которой н
ITER 5000: score=14367.023 | матрцшка русская деревянная игрушка в виде расписной куклы внутри которой н
ITER 6000: score=14367.023 | матрцшка русская деревянная игрушка в виде расписной куклы внутри которой н
ITER 7000: score=14367.023 | матрцшка русская деревянная игрушка в виде расписной куклы внутри которой н
ITER 8000: score=14367.023 | матрцшка русская деревянная игрушка в виде расписной куклы внутри которой н
ITER 9000: score=14367.023 | матрцшка русская деревянная игр

In [99]:
decrypted_text

'матрцшка русская деревянная игрушка в виде расписной куклы внутри которой находятся подобные ей куклы меньшего размера число вложенных кукол обычно три и более обычно имеют эорму яйфа с плоским донфем и состоят из двух разёцмных частей верхней и нижней на традифионных матрцшках изображена женщина в красном сараэане и платке в наше время темы для росписи разнообразны сказочные персонажи девушки а также семьи стали нередки и матрцшки пародийного характера с изображением политических деятелей сравнительно недавно стала набирать популярность матрцшка с изображением портрета портретная матрцшка сейчас матрцшки делают в различных мастерских сначала подбирают подходящий вид древесины изза мягкости в основном выбирают липу реже ольху или берцзу деревья обычно срубают ранней весной снимают кору но не полностью чтобы во время сушки древесина не давала трещин затем брцвна складируют и сушат в течение нескольких лет в хорошо вентилируемом месте древесина может заготавливаться в регионе изготовлен

Наш метод прекрасно справляется с тестовыми текстами и уже меньше, чем за 10к итераций сходится. Точность расшифровки > 98%, алгоритм ошибается лишь на редких буквах (например ё).

Теперь попробуем расшифровать сообщение из задания:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙
⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴
↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

In [120]:
decryption_rule, decrypted_text = MCMC_decryption(30000, test, bigram_freq_ru, 2, ru)

ITER 0: score=983.93 | ыхижпбзпбжджтыпоемнришозвпжижпяейтжпоемнришозвптылхтпгп течепхеекуыожьплете
ITER 1000: score=1733.625 | осли вы викино ремчалдрый или бегни ремчалдрый нотсн у ьнеже сеехпория тене
ITER 2000: score=1735.903 | осли ты типино ремчалкрый или бегни ремчалкрый новсн у ьнеде сеешхория вене
ITER 3000: score=1734.326 | осли ты типино ревзалкрый или бегни ревзалкрый номсн у ьнеде сеехшория мене
ITER 4000: score=1736.918 | осли ты типино ревзалкрый или дегни ревзалкрый номсн у ьнеже сеехшория мене
ITER 5000: score=1737.987 | осли ты типино ревзалкрый или дегни ревзалкрый номсн у ьнебе сеехшория мене
ITER 6000: score=1741.164 | осли ты типино ревзалкрый или дегни ревзалкрый номсн у ьнеже сеебхория мене
ITER 7000: score=1741.164 | осли ты типино ревзалкрый или дегни ревзалкрый номсн у ьнеже сеебхория мене
ITER 8000: score=1741.164 | осли ты типино ревзалкрый или дегни ревзалкрый номсн у ьнеже сеебхория мене
ITER 9000: score=1741.164 | осли ты типино ревзалкрый или дегни ревза

In [121]:
decrypted_text

'если вы вимите норжальный или подти норжальный текст у чтого сообшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса хотя конедно я нидего не обешаю'

Конечно из-за того, что размер сообщения не очень большой, качество расшифровки не идеальное, но предложение читаемое, что очень приятно :)